In [1]:
from dotenv import load_dotenv
load_dotenv()
import os

NOTE: This is ONLY necessary in jupyter notebook.
Details: Jupyter runs an event-loop behind the scenes. 
This results in nested event-loops when we start an event-loop to make async queries.
This is normally not allowed, we use nest_asyncio to allow it for convenience.

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.ERROR)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Global Models


In [4]:
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
Settings.llm = OpenAI(model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [5]:
# Using the LlamaDebugHandler to print the trace of the sub questions
# captured by the SUB_QUESTION callback event type
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

In [6]:
# %pip install llama-index-readers-file
# %pip install llama-index-llms-openai

In [7]:
from pydantic import BaseModel
from unstructured.partition.html import partition_html
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

# Perform Data Extraction
In these sections we use Unstructured to parse out the table and non-table elements.

In [8]:
# this wan't working.. DIdn't care to look at it. but rather downloaded the files from browser.
 
#!curl "https://www.dropbox.com/scl/fi/mlaymdy1ni1ovyeykhhuk/tesla_2021_10k.htm?rlkey=qf9k4zn0ejrbm716j0gg7r802&dl=1" -o "tesla_2021_10k.htm"
#!curl "https://www.dropbox.com/scl/fi/rkw0u959yb4w8vlzz76sa/tesla_2020_10k.htm?rlkey=tfkdshswpoupav5tqigwz1mp7&dl=1" -o "tesla_2020_10k.htm"

# Extract Elements
We use Unstructured to extract table and non-table elements from the 10-K filing.

In [9]:
from llama_index.readers.file import FlatReader
from pathlib import Path

reader = FlatReader()
docs_2021 = reader.load_data(Path("tesla_2021_10k.htm"))
docs_2020 = reader.load_data(Path("tesla_2020_10k.htm"))

In [10]:
from llama_index.core.node_parser import UnstructuredElementNodeParser
node_parser = UnstructuredElementNodeParser()

In [11]:
import os
import pickle

if not os.path.exists("2021_nodes.pkl"):
    raw_nodes_2021 = node_parser.get_nodes_from_documents(docs_2021)
    pickle.dump(raw_nodes_2021, open("2021_nodes.pkl", "wb"))
else:
    raw_nodes_2021 = pickle.load(open("2021_nodes.pkl", "rb"))

In [12]:
base_nodes_2021, node_mappings_2021 = node_parser.get_base_nodes_and_mappings(
    raw_nodes_2021
)

In [13]:
base_nodes_2021

[TextNode(id_='ddff60ff-775a-45cf-b2cf-46575225ef1f', embedding=None, metadata={'filename': 'tesla_2021_10k.htm', 'extension': '.htm'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='6fab829c-e2ab-4884-93fb-cc54cb7735c8', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'filename': 'tesla_2021_10k.htm', 'extension': '.htm'}, hash='ec3c267a9a393f71ed8d902be229213ccdcc47059414b56d723bd35b2f59cfbc'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='3dcd394a-381b-4e99-933f-612563b66337', node_type=<ObjectType.INDEX: '3'>, metadata={'col_schema': 'Column: Financial Metrics\nType: numeric\nSummary: Key financial metrics including revenue, profit, and expenses.\n\nColumn: Year\nType: date\nSummary: The fiscal year for which the financial metrics are reported.', 'filename': 'tesla_2021_10k.htm', 'extension': '.htm'}, hash='526b8dd6feede4c6448ff856dd231da60f387b07323cc9bda99b75a93a2d956f')}, text='UNIT

In [14]:
node_mappings_2021

{'d256855c-199b-433d-ae26-c6a60138d70d': TextNode(id_='d256855c-199b-433d-ae26-c6a60138d70d', embedding=None, metadata={'filename': 'tesla_2021_10k.htm', 'extension': '.htm', 'table_df': "{'': {0: 'ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934'}}", 'table_summary': 'This table presents the annual financial performance and key metrics of the company as required by the Securities Exchange Act of 1934.,\nwith the following table title:\nANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934,\nwith the following columns:\n- Financial Metrics: Key financial metrics including revenue, profit, and expenses.\n- Year: The fiscal year for which the financial metrics are reported.\n'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='6fab829c-e2ab-4884-93fb-cc54cb7735c8', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'filename': 'tesla_2021_

In [ ]:
example_index_node = [b for b in base_nodes_2021 if isinstance(b, IndexNode)][
    20
]

# Index Node
print(
    f"\n--------\n{example_index_node.get_content(metadata_mode='all')}\n--------\n"
)
# Index Node ID
print(f"\n--------\nIndex ID: {example_index_node.index_id}\n--------\n")
# Referenceed Table
print(
    f"\n--------\n{node_mappings_2021[example_index_node.index_id].get_content()}\n--------\n"
)

# Construct Base Query Engine


In [15]:
from llama_index.core import VectorStoreIndex

In [16]:
# construct top-level vector index + query engine
vector_index = VectorStoreIndex(base_nodes_2021)
vector_query_engine = vector_index.as_query_engine(similarity_top_k=1)

**********
Trace: index_construction
    |_embedding -> 4.973764 seconds
    |_embedding -> 2.020485 seconds
**********


In [18]:
# Results from BASE query Engine
response = vector_query_engine.query("What was the revenue in 2021?")
print(str(response))

**********
Trace: query
    |_query -> 2.320962 seconds
      |_retrieve -> 0.585847 seconds
        |_embedding -> 0.562848 seconds
      |_synthesize -> 1.733115 seconds
        |_templating -> 0.0 seconds
        |_llm -> 1.727115 seconds
**********
The total revenue in 2021 included significant contributions from various segments: automotive sales revenue increased by $19.52 billion, automotive leasing revenue rose by $590 million, services and other revenue grew by $1.50 billion, and energy generation and storage revenue increased by $795 million. Each of these segments contributed to the overall revenue growth compared to 2020.


# Setup Recursive Retriever and Query Engine
Now that we've extracted tables and their summaries, we can setup a recursive retriever in LlamaIndex to query these tables.

In [19]:
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

In [20]:
vector_retriever = vector_index.as_retriever(similarity_top_k=1)

In [21]:
recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    node_dict=node_mappings_2021,
    verbose=True,
)
query_engine = RetrieverQueryEngine.from_args(recursive_retriever)

# Run some Queries


In [22]:
response = query_engine.query("What was the revenue in 2021?")

Retrieving with query id None: What was the revenue in 2021?
Retrieving text node: Services and other revenue consists of non-warranty after-sales vehicle services, sales of used vehicles, retail merchandise, sales by our acquired subsidiaries to third party customers and vehicle insurance revenue.

2021 compared to 2020

Automotive sales revenue increased $19.52 billion, or 79%, in the year ended December 31, 2021 as compared to the year ended December 31, 2020, primarily due to an increase of 433,815 Model 3 and Model Y cash deliveries year over year from production ramping at both Gigafactory Shanghai and the Fremont Factory at a slightly higher combined average selling price from a higher proportion of Model Y sales offset by regional sales mix. Additionally, we had a $365 million net release of our sales return reserve on vehicles sold with resale value guarantees, which increased our automotive sales revenue, due to actual return rates being lower than expected and increases in r

In [23]:
print(str(response))

The total revenue in 2021 included significant increases in various segments: automotive sales revenue increased by $19.52 billion, automotive leasing revenue rose by $590 million, services and other revenue grew by $1.50 billion, and energy generation and storage revenue increased by $795 million. The specific total revenue figure for 2021 is not provided in the information.


# Recursive

In [24]:
response = query_engine.query("What were the total cash flows in 2021?")
print(str(response))

Retrieving with query id None: What were the total cash flows in 2021?
Retrieving text node: government and other marketable securities, digital assets and providing product related financing. In the first quarter of 2021, we invested an aggregate $1.50 billion in bitcoin. The fair market value of our bitcoin holdings as of December 31, 2021 was $1.99 billion. We believe in the long-term potential of digital assets both as an investment and also as a liquid alternative to cash. As with any investment and consistent with how we manage fiat-based cash and cash equivalent accounts, we may increase or decrease our holdings of digital assets at any time based on the needs of the business and our view of market and environmental conditions. However, digital assets may be subject to volatile market prices, which may be unfavorable at the times when we may want or need to liquidate them. Additionally, we held short-term marketable securities of $131 million as of December 31, 2021.

42

Summar

# Base

In [25]:
response = vector_query_engine.query("What were the total cash flows in 2021?")
print(str(response))

**********
Trace: query
    |_query -> 1.992863 seconds
      |_retrieve -> 0.522605 seconds
        |_embedding -> 0.493614 seconds
      |_synthesize -> 1.469258 seconds
        |_templating -> 0.0 seconds
        |_llm -> 1.454262 seconds
**********
The total cash flows in 2021 can be summarized as follows:

- Net cash provided by operating activities: $11,497 million
- Net cash used in investing activities: $(7,868) million
- Net cash used in financing activities: $(5,203) million

To calculate the total cash flows, we can add these figures together:

Total cash flows = $11,497 million - $7,868 million - $5,203 million = $(574) million

Therefore, the total cash flows in 2021 were $(574) million.


In [ ]:
response = query_engine.query("What are the risk factors for Tesla?")


Retrieving with query id None: What are the risk factors for Tesla?
Retrieving text node: For example, we are developing self-driving and driver assist technologies to rely on vision-based sensors, unlike alternative technologies in development that additionally require other redundant sensors. There is no guarantee that any incremental changes in the specific equipment we deploy in our vehicles over time will not result in initial functional disparities from prior iterations or will perform as expected in the timeframe we anticipate, or at all.

Our products are also highly dependent on software, which is inherently complex and may contain latent defects or errors or be subject to external attacks. Issues experienced by our customers have included those related to the Model S and Model X 17-inch display screen, the panoramic roof and the 12-volt battery in the Model S, the seats and doors in the Model X and the operation of solar panels installed by us. Although we attempt to remedy a

In [27]:
print(str(response))

Tesla faces several risk factors, including:

1. **Product Reliability and Software Complexity**: The company relies heavily on software, which can be complex and may contain defects or errors. There is a risk that issues may not be detected or resolved before products reach customers, potentially affecting quality and performance.

2. **Product Liability Claims**: The automobile industry is prone to significant product liability claims. Tesla's vehicles, particularly those with Autopilot or Full Self-Driving features, may be involved in accidents, leading to claims and regulatory scrutiny. The company self-insures against these risks, which could result in substantial financial burdens.

3. **Public Credibility and Confidence**: Maintaining credibility among customers, investors, and other stakeholders is crucial for Tesla's success. Factors such as competition, operational challenges, and negative public perceptions can impact confidence in the company's long-term viability.

4. **Fi

In [ ]:
response = vector_query_engine.query("What are the risk factors for Tesla?")

**********
Trace: query
    |_query -> 3.318357 seconds
      |_retrieve -> 0.91558 seconds
        |_embedding -> 0.902582 seconds
      |_synthesize -> 2.400771 seconds
        |_templating -> 0.0 seconds
        |_llm -> 2.392769 seconds
**********


In [29]:
print(str(response))

Tesla faces several risk factors, including:

1. **Product Reliability and Software Complexity**: The company relies heavily on software, which can be complex and may contain defects or errors. There is uncertainty regarding the long-term quality and performance of their products, and issues have been reported with various vehicle components.

2. **Product Liability Claims**: The automobile industry is prone to significant product liability claims. Tesla's vehicles, particularly those with Autopilot or Full Self-Driving features, may be involved in accidents, leading to claims and regulatory scrutiny. Additionally, the risk of battery failures poses potential safety hazards.

3. **Public Credibility and Confidence**: Maintaining credibility among customers, investors, and other stakeholders is crucial for Tesla's success. Factors such as competition, production delays, and negative public perceptions can impact confidence in the company's long-term viability.

4. **Financing and Compli

# Try Table Comparisons
In this setting we load in both the 2021 and 2020 10K filings, parse each into a hierarchy of tables/text objects, define a recursive retriever over each, and then compose both with a SubQuestionQueryEngine.

This allows us to execute document comparisons against both.

# Define E2E Recursive Retriever Function


In [30]:
import pickle
import os


def create_recursive_retriever_over_doc(docs, nodes_save_path=None):
    """Big function to go from document path -> recursive retriever."""
    node_parser = UnstructuredElementNodeParser()
    if nodes_save_path is not None and os.path.exists(nodes_save_path):
        raw_nodes = pickle.load(open(nodes_save_path, "rb"))
    else:
        raw_nodes = node_parser.get_nodes_from_documents(docs)
        if nodes_save_path is not None:
            pickle.dump(raw_nodes, open(nodes_save_path, "wb"))

    base_nodes, node_mappings = node_parser.get_base_nodes_and_mappings(
        raw_nodes
    )

    ### Construct Retrievers
    # construct top-level vector index + query engine
    vector_index = VectorStoreIndex(base_nodes)
    vector_retriever = vector_index.as_retriever(similarity_top_k=2)
    recursive_retriever = RecursiveRetriever(
        "vector",
        retriever_dict={"vector": vector_retriever},
        node_dict=node_mappings,
        verbose=True,
    )
    query_engine = RetrieverQueryEngine.from_args(recursive_retriever)
    return query_engine, base_nodes

# Create Sub Question Query Engine


In [31]:
import nest_asyncio

nest_asyncio.apply()

In [32]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

In [35]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o-mini")

In [36]:
query_engine_2021, nodes_2021 = create_recursive_retriever_over_doc(
    docs_2021, nodes_save_path="2021_nodes.pkl"
)
query_engine_2020, nodes_2020 = create_recursive_retriever_over_doc(
    docs_2020, nodes_save_path="2020_nodes.pkl"
)

**********
Trace: index_construction
    |_embedding -> 4.569119 seconds
    |_embedding -> 1.649401 seconds
**********


6it [00:00, ?it/s]

**********
Trace: index_construction
    |_node_parsing -> 0.001998 seconds
      |_chunking -> 0.000999 seconds
**********


**********
Trace: index_construction
    |_node_parsing -> 0.000999 seconds
      |_chunking -> 0.000999 seconds
**********
**********
Trace: index_construction
    |_node_parsing -> 0.002999 seconds
      |_chunking -> 0.001001 seconds
**********
**********
Trace: index_construction
    |_node_parsing -> 0.004 seconds
      |_chunking -> 0.001 seconds
**********
**********
Trace: query
    |_query -> 0.0 seconds
      |_synthesize -> 0.0 seconds
        |_llm -> 0.0 seconds
**********
**********
Trace: index_construction
    |_node_parsing -> 0.000996 seconds
      |_chunking -> 0.0 seconds
**********
**********
Trace: query
    |_query -> 0.0 seconds
      |_synthesize -> 0.0 seconds
        |_llm -> 0.0 seconds
**********
**********
Trace: index_construction
    |_node_parsing -> 0.001001 seconds
      |_chunking -> 0.001001 seconds
**********
**********
Trace: query
    |_query -> 0.0 seconds
      |_synthesize -> 0.0 seconds
        |_llm -> 0.0 seconds
**********
**********
Trace

c:\Code\Github\LlamaIndex\venv\lib\site-packages\llama_index\core\node_parser\relational\base_element.py:420: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  str(table_df.to_dict())
c:\Code\Github\LlamaIndex\venv\lib\site-packages\llama_index\core\node_parser\relational\base_element.py:420: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  str(table_df.to_dict())
c:\Code\Github\LlamaIndex\venv\lib\site-packages\llama_index\core\node_parser\relational\base_element.py:420: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  str(table_df.to_dict())
c:\Code\Github\LlamaIndex\venv\lib\site-packages\llama_index\core\node_parser\relational\base_element.py:420: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  str(table_df.to_dict())
c:\Code\Github\LlamaIndex\venv\lib\site-packages\llama_index\core\node_parser\relational\base_element.py:420: UserWarning: DataFrame columns are not

**********
Trace: index_construction
    |_embedding -> 11.303066 seconds
**********


In [37]:
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine_2021,
        metadata=ToolMetadata(
            name="tesla_2021_10k",
            description=(
                "Provides information about Tesla financials for year 2021"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=query_engine_2020,
        metadata=ToolMetadata(
            name="tesla_2020_10k",
            description=(
                "Provides information about Tesla financials for year 2020"
            ),
        ),
    ),
]

sub_query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    llm=llm,
    use_async=True,
)

# Try out some Comparisons


In [38]:
response = sub_query_engine.query(
    "Can you compare and contrast the cash flow in 2021 with 2020?"
)

Generated 8 sub questions.
[tesla_2021_10k] Q: What was Tesla's cash flow from operating activities in 2021?
Retrieving with query id None: What was Tesla's cash flow from operating activities in 2021?
Retrieving text node: 299 — — 2,299 — 2,299 Contributions from noncontrolling interests 2 — — — — — — — — Distributions to noncontrolling interests ( 66 ) — — — — — — ( 106 ) ( 106 ) Buy-outs of noncontrolling interests ( 15 ) — — 5 — — 5 — 5 Net income 43 — — — — 5,519 5,519 82 5,601 Other comprehensive loss — — — — ( 309 ) — ( 309 ) — ( 309 ) Balance as of December 31, 2021 $ 568 1,033 $ 1 $ 29,803 $ 54 $ 331 $ 30,189 $ 826 $ 31,015

The accompanying notes are an integral part of these consolidated financial statements.

52

Tesla, Inc.

Consolidated Statements of Cash Flows

(in millions)

Year Ended December 31, 2021 2020 2019 Cash Flows from Operating Activities Net income (loss) $ 5,644 $ 862 $ ( 775 ) Adjustments to reconcile net income (loss) to net cash provided by operating act

In [39]:
print(str(response))

In 2021, Tesla's cash flow from operating activities significantly increased to $11.50 billion, compared to $5.943 billion in 2020. This indicates a strong improvement in operational performance.

However, cash flow from investing activities showed a notable increase in outflow, with a net cash outflow of $7.868 billion in 2021, compared to a smaller outflow of $(3.132) billion in 2020. This suggests that Tesla invested more heavily in its growth and expansion in 2021.

In terms of financing activities, there was a shift from a positive cash flow of $9.973 billion in 2020 to a negative cash flow of $(5.203) billion in 2021. This change indicates that Tesla may have reduced its reliance on external financing or paid down debt in 2021.

Overall, while operating cash flow improved, the increased outflow from investing activities and the shift to negative financing activities resulted in total cash flows of $(1.574) billion in 2021, contrasting with a positive total cash flow of $12.784 bi

In [40]:
response = sub_query_engine.query(
    "Can you compare and contrast the R&D expenditures in 2021 vs. 2020?"
)

Generated 2 sub questions.
[tesla_2021_10k] Q: What were Tesla's R&D expenditures in 2021?
Retrieving with query id None: What were Tesla's R&D expenditures in 2021?
Retrieving text node: Gross margin for energy generation and storage decreased from 0.9% in the year ended December 31, 2020 to -4.6% in the year ended December 31, 2021, primarily due to a higher proportion of Solar Roof in our overall energy business which operated at a lower gross margin as a result of temporary manufacturing underutilization during product ramp despite an improvement in gross margin compared to the prior year and lower Powerpack gross margin as fixed cost absorptions are negatively impacted as we phase out the product following the introduction of Megapack. These decreases were partially offset by a higher proportion of Powerwall in our overall energy business which operated at a higher gross margin as well as an increase in Megapack gross margin as we continue to scale the products.

Research and Deve

In [41]:
print(str(response))

In 2021, Tesla's R&D expenditures were $2.593 billion, which represents a significant increase compared to the $1.491 billion spent in 2020. This indicates a growth of approximately 74% in R&D spending from 2020 to 2021, reflecting a stronger investment in research and development efforts during that period.


In [42]:
response = sub_query_engine.query(
    "Can you compare and contrast the risk factors in 2021 vs. 2020?"
)

Generated 2 sub questions.
[tesla_2021_10k] Q: What are the risk factors listed in Tesla's 2021 10-K report?
Retrieving with query id None: What are the risk factors listed in Tesla's 2021 10-K report?
Retrieving text node: If an emerging growth company, indicate by check mark if the registrant has elected not to use the extended transition period for complying with any new or revised financial accounting standards provided pursuant to Section 13(a) of the Exchange Act. ☐

Indicate by check mark whether the Registrant has filed a report on and attestation to its management’s assessment of the effectiveness of its internal control over financial reporting under Section 404(b) of the Sarbanes-Oxley Act (15 U.S.C. 7262(b)) by the registered public accounting firm that prepared or issued its audit report.

Indicate by check mark whether the registrant is a shell company (as defined in Rule 12b-2 of the Exchange Act). Yes No ☒

The aggregate market value of voting stock held by non-affiliat

In [43]:
print(str(response))

The risk factors listed in Tesla's 2021 10-K report and those in the 2020 report share several similarities, primarily revolving around the impacts of the global COVID-19 pandemic on operations and supply chains. Both years highlight macroeconomic conditions as a significant risk, emphasizing how the pandemic has influenced consumer trends and operational capabilities.

In 2020, the focus was on temporary suspensions of operations at manufacturing facilities and challenges in vehicle deliveries due to reduced operations across various departments. The report also noted increased expenses from port congestion and supplier shutdowns, along with a shortfall in microchip supply affecting production.

By 2021, the risk factors evolved to include more specific challenges such as semiconductor shortages and labor shortages, indicating a continuation and intensification of supply chain issues. The 2021 report also emphasized the need for stable operations and government cooperation, reflecting

# Try Comparing against Baseline


In [44]:
vector_index_2021 = VectorStoreIndex(nodes_2021)
vector_query_engine_2021 = vector_index_2021.as_query_engine(
    similarity_top_k=2
)
vector_index_2020 = VectorStoreIndex(nodes_2020)
vector_query_engine_2020 = vector_index_2020.as_query_engine(
    similarity_top_k=2
)
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine_2021,
        metadata=ToolMetadata(
            name="tesla_2021_10k",
            description=(
                "Provides information about Tesla financials for year 2021"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=vector_query_engine_2020,
        metadata=ToolMetadata(
            name="tesla_2020_10k",
            description=(
                "Provides information about Tesla financials for year 2020"
            ),
        ),
    ),
]

base_sub_query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    llm=llm,
    use_async=True,
)

**********
Trace: index_construction
    |_embedding -> 6.220957 seconds
    |_embedding -> 1.136796 seconds
**********
**********
Trace: index_construction
    |_embedding -> 4.835362 seconds
**********


In [ ]:
response = base_sub_query_engine.query(
    "Can you compare and contrast the cash flow in 2021 with 2020?"
)


Generated 8 sub questions.
[tesla_2021_10k] Q: What was Tesla's cash flow from operating activities in 2021?
[tesla_2020_10k] Q: What was Tesla's cash flow from operating activities in 2020?
[tesla_2021_10k] Q: What was Tesla's cash flow from investing activities in 2021?
[tesla_2020_10k] Q: What was Tesla's cash flow from investing activities in 2020?
[tesla_2021_10k] Q: What was Tesla's cash flow from financing activities in 2021?
[tesla_2020_10k] Q: What was Tesla's cash flow from financing activities in 2020?
[tesla_2021_10k] Q: What was Tesla's net cash flow in 2021?
[tesla_2020_10k] Q: What was Tesla's net cash flow in 2020?
[tesla_2020_10k] A: Tesla's cash flow from financing activities in 2020 was $9,973 million.
[tesla_2021_10k] A: Tesla's cash flow from financing activities in 2021 was $(5,203) million.
[tesla_2021_10k] A: Tesla's cash flow from investing activities in 2021 was a net cash outflow of $7.868 billion.
[tesla_2021_10k] A: Tesla's cash flow from operating activiti

In [46]:
print(str(response))

In 2021, Tesla's cash flow from operating activities significantly increased to $11.50 billion compared to $5.943 billion in 2020, indicating strong operational performance. However, cash flow from investing activities showed a notable net cash outflow of $7.868 billion in 2021, which was a larger outflow than the $(3,132) million recorded in 2020. This suggests that Tesla invested more heavily in 2021.

Regarding financing activities, there was a shift from a positive cash flow of $9.973 million in 2020 to a negative cash flow of $(5,203) million in 2021, indicating that Tesla reduced its reliance on external financing or paid down debt.

Overall, while operating cash flow improved, the increased investment outflows and the shift in financing activities resulted in a net cash flow decrease of $574 million in 2021, contrasting with a positive net cash flow of $12,784 million in 2020. This reflects a more aggressive investment strategy in 2021 despite the decline in overall net cash flo